In [2]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sc
import scipy.special  # Needed for old SciPy versions

The radial Schrödinger equation (SE) for the hydrogen atom can be found from the separation Ansatz
$$
\psi(\vec{r}) = R_{n \ell}(r) Y_{\ell}^{m}(\theta, \varphi).
$$
Also see https://en.wikipedia.org/wiki/Hydrogen_atom#Wavefunction.

If we further set $R_{n \ell}(r) = \frac{u_{n \ell}(r)}{r}$, the SE becomes quite simple
\begin{equation}
    \left[-\frac{1}{2}\frac{\mathrm{d}^2}{\mathrm{d}r^2} - \frac{1}{r} + \frac{\ell(\ell+1)}{2 r^2} \right]u_{n \ell}(r) = E_{n \ell} u_{n \ell}(r).
\end{equation}
In this form, it is similar to the 1D case we considered before, and we can (basically) use the same algorithm to solve it.

When it comes to the analytical solutions, things do not look as simple anymore. For the radial part, we get the following terms:
$$
R_{n \ell}(r) = \sqrt{\left( \frac{2}{n} \right)^3 \frac{(n-\ell-1)!}{2n(n+\ell)!}} e^{-\frac{r}{2}} \left( \frac{2r}{n} \right)^{\ell} L_{n-\ell-1}^{2\ell+1}\left(\frac{2r}{n}\right).
$$
This function has already been implemented below and will be used for comparison with the numerical results. The eigenenergies are
$$
E_{n \ell} = -\frac{1}{2 n^2}
$$
with $n=1, 2, \ldots$ and $\ell=0, \ldots, n-1$.

In [3]:
def radial_function(n, l, r):
    """See https://en.wikipedia.org/wiki/Hydrogen_atom#Wavefunction."""
    rho = 2 * r / n
    L = sc.special.genlaguerre(n - l - 1, 2 * l + 1)
    prefactor = np.sqrt((2 / n) ** 3 * sc.special.factorial(n - l - 1) / (2 * n * sc.special.factorial(n + l)))
    return prefactor * np.exp(-rho / 2) * rho**l * L(rho)

## Radial dependence

In [4]:
# Import the prepared function for the radial SE
from numerov import solve_schroedinger_radial

In [ ]:
# Input parameters
N = 2000  # Number of grid points
xmax = 40  # Extent of the grid

# Preparation of the grid
rs = np.linspace(0, xmax, N)
rs[0] = 1e-6  # Avoid dividing by zero
h = rs[1] - rs[0]
# ...and the potential
V = -1 / rs  # Coulomb potential

In [ ]:
k = 1  # Radial quantum number (= number of nodes),
# The principal quantum number is n = k + 1 + ell
ell = 0  # Angular quantum number
Etry = -1

# Use Numerov to obtain the unnormalized wave function
u, E, dE, n_nodes = solve_schroedinger_radial(V, ell, k, h, Etry=Etry)
print(f"tried E={Etry:.1f} got n_nodes={n_nodes} and E={E:.1f} (expectation was n_nodes={k} and E={-0.5 / (k + 1 + ell) ** 2:.1f})")

# ...and normalize it
u /= np.sqrt(np.trapezoid(u**2, dx=h))

In [ ]:
# Plot results
fix, axs = plt.subplots(2, 1, sharex=True)

# ...first the potential
axs[0].plot(rs[1:], V[1:], label="Coulomb", color="black")
axs[0].plot(
    rs[1:],
    V[1:] + ell * (ell + 1) / (2 * rs[1:] ** 2),
    label="eff. potential",
    color="green",
    linestyle="dashed",
)
axs[0].axhline(y=E, color="grey")
axs[0].set_ylabel(r"$V$")
axs[0].set_ylim((-0.6, 0.1))

# ...and then the wave function
axs[1].plot(rs, u, label=rf"$E_{{{k + 1} {ell}}}={E:.1f}$")
axs[1].plot(
    rs,
    rs * radial_function(k + 1 + ell, ell, rs),
    label="analytic",
    color="orange",
    linestyle="dashed",
)
axs[1].set_xlabel(r"$r$")
axs[1].set_ylabel(r"$u$")

plt.legend()
plt.tight_layout()
plt.show()

<div class="alert alert-block alert-info">
<b>Try</b> to calculate and plot the radial probability density $4\pi r^2 |R_{n \ell}(r)|^2$ for different states.
</div>

## Angular dependence

For now, we have only taken a look at the radial part of the wave function, neglecting the angular contribution. The angular part looks as follows:
$$
Y_{\ell}^{m}(\theta, \varphi) = (-1)^m \sqrt{\frac{(2\ell+1)}{4\pi}\frac{(\ell-m)!}{(\ell+m)!}} P_{\ell}^{m}(\cos\theta) e^{im\varphi}.
$$

In [ ]:
def angular_function(m, l, theta, phi):
    """See https://en.wikipedia.org/wiki/Hydrogen_atom#Wavefunction."""
    P = sc.special.lpmv(m, l, np.cos(theta))
    prefactor = (-1) ** m * np.sqrt((2 * l + 1) * sc.special.factorial(l - np.abs(m)) / (4 * np.pi * sc.special.factorial(l + np.abs(m))))
    return prefactor * P * np.exp(1j * m * phi)

<div class="alert alert-block alert-info">
<b>Take</b> a look at the hydrogen wave functions in the $x$-$z$ plane. What happens to increasing $n$? Can you identify orbitals you already know?

<p>
<b>Bonus:</b> Implement a check that ensures that only valid combinations of quantum numbers are allowed.
</p>
</div>

In [ ]:
def hydrogen_probability_density(n, l, m):
    """Calculate hydrogen probability densities."""
    # Sample the x-z plane
    z = x = np.linspace(-50, 50, 500)
    z, x = np.meshgrid(z, x)

    # Transform coordinates
    r = np.sqrt(x**2 + z**2)
    theta = np.arctan2(x, z)
    phi = 0

    # Calculate the wave function
    psi = radial_function(n, l, r) * angular_function(m, l, theta, phi)

    # Return the probability density, do not forget about the volume element!
    return np.abs(psi) ** 2 * r**2 * np.cos(phi)

In [ ]:
def plot_hydrogen(n, l, m):
    """Plot the hydrogen probability densities.

    Args:
        n (int): Principal quantum number.
        l (int): Orbital quantum number.
        m (int): Magnetic quantum number.
    """
    fig, ax = plt.subplots(figsize=(12.5, 12.5))
    ax.imshow(hydrogen_probability_density(n, l, m))
    plt.title(rf"$|\psi_{{{n}{l}{m}}}(r, \theta, \varphi)|^2$", size=30)
    plt.xticks([], [])
    plt.yticks([], [])
    plt.show()

In [ ]:
plot_hydrogen(4, 0, 0)